In [1]:
import numpy as np
import pandas as pd

In [2]:
eps = np.finfo(float).eps
eps

2.220446049250313e-16

Here eps is the smallest number At times when we gey log 0 we use eps

In [3]:
df = pd.read_csv("play_tennis.csv")

In [4]:
df.values

array([['D1', 'Sunny', 'Hot', 'High', 'Weak', 'No'],
       ['D2', 'Sunny', 'Hot', 'High', 'Strong', 'No'],
       ['D3', 'Overcast', 'Hot', 'High', 'Weak', 'Yes'],
       ['D4', 'Rain', 'Mild', 'High', 'Weak', 'Yes'],
       ['D5', 'Rain', 'Cool', 'Normal', 'Weak', 'Yes'],
       ['D6', 'Rain', 'Cool', 'Normal', 'Strong', 'No'],
       ['D7', 'Overcast', 'Cool', 'Normal', 'Strong', 'Yes'],
       ['D8', 'Sunny', 'Mild', 'High', 'Weak', 'No'],
       ['D9', 'Sunny', 'Cool', 'Normal', 'Weak', 'Yes'],
       ['D10', 'Rain', 'Mild', 'Normal', 'Weak', 'Yes'],
       ['D11', 'Sunny', 'Mild', 'Normal', 'Strong', 'Yes'],
       ['D12', 'Overcast', 'Mild', 'High', 'Strong', 'Yes'],
       ['D13', 'Overcast', 'Hot', 'Normal', 'Weak', 'Yes'],
       ['D14', 'Rain', 'Mild', 'High', 'Strong', 'No']], dtype=object)

In [5]:
df.keys()

Index(['day', 'outlook', 'temp', 'humidity', 'wind', 'play'], dtype='object')

In [6]:
target = df.keys()[-1]
target

'play'

In [7]:
df.keys()[:-1]

Index(['day', 'outlook', 'temp', 'humidity', 'wind'], dtype='object')

In [8]:
df[target].unique()

array(['No', 'Yes'], dtype=object)

In [9]:
df[target].value_counts()

Yes    9
No     5
Name: play, dtype: int64

# Function to calculate entropy

In [10]:
def find_entropy_whole(df):
    target = df.keys()[-1]
    
    overall_entropy = 0
    
    values_in_target,count_of_values = np.unique(df[target],return_counts=True)
    
    probabilites_of_values = count_of_values/np.sum(count_of_values)
    
    return -1*np.sum(probabilites_of_values*np.log2(probabilites_of_values))

In [11]:
find_entropy_whole(df)

0.9402859586706311

# Calculating entropy of each attribute

In [55]:
def find_entropy_of_attribute(df,attribute):
    target = df.keys()[-1]
    
    values_in_target = np.unique(df[target])
    
    values_in_attribute = np.unique(df[target])
    
    entropy_attribute = 0
    
    for value_in_attribute in values_in_attribute:
        overall_entropy = 0
        for value_in_target in values_in_target:
            
            num = len(df[attribute][df[attribute]==value_in_attribute][df[target]==value_in_target])
            den = len(df[attribute][df[attribute]==value_in_attribute])
            
            p = num/(den+eps)
            overall_entropy += -p*np.log2(p+eps)
        p2 = den/len(df)
        entropy_attribute += -p2*overall_entropy
        
        return abs(entropy_attribute)

In [56]:
find_entropy_of_attribute(df,df.keys()[2])

0.0

In [45]:
def find_entropy_of_attribute(df,attribute):
    target = df.keys()[-1]
    
    values_in_target = np.unique(df[target])
    
    values_in_attribute = np.unique(df[attribute])
    attribute_entropy = 0
    
    for value_in_attribute in values_in_attribute:
        overall_entropy = 0
        for value_in_target in values_in_target:
            num = len(df[df[attribute]==value_in_attribute][df[target]==value_in_target])
            den = len(df[attribute][df[attribute]==value_in_attribute])
            
            p = num/(den+eps)
            overall_entropy += -p*np.log2(p+eps)
           
        p2 = den/len(df)
        attribute_entropy += p2*overall_entropy
        
    return abs(attribute_entropy)

In [46]:
find_entropy_of_attribute(df,df.keys()[1])

C:\Users\user\AppData\Local\Temp/ipykernel_9980/246884125.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  num = len(df[df[attribute]==value_in_attribute][df[target]==value_in_target])


0.6935361388961914

In [49]:
for i_attribute in df.keys()[1:-1]:
    print(f'The Entropy of {i_attribute} is: {find_entropy_of_attribute(df,i_attribute)}')

The Entropy of outlook is: 0.6935361388961914
The Entropy of temp is: 0.9110633930116756
The Entropy of humidity is: 0.7884504573082889
The Entropy of wind is: 0.892158928262361


C:\Users\user\AppData\Local\Temp/ipykernel_9980/246884125.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  num = len(df[df[attribute]==value_in_attribute][df[target]==value_in_target])


# Finding the best attribute

In [50]:
def find_best_attribute_to_divide(df):
    IG = []
    
    all_attribute_names = df.keys()[1:-1]
    
    for attribute in all_attribute_names:
        IG.append(find_entropy_whole(df) - find_entropy_of_attribute(df,attribute))
        
    index_of_attribute_with_max_entropy = np.argmax(IG)
    
    best_attribute = all_attribute_names[index_of_attribute_with_max_entropy]
    
    return best_attribute

In [51]:
find_best_attribute_to_divide(df)

C:\Users\user\AppData\Local\Temp/ipykernel_9980/246884125.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  num = len(df[df[attribute]==value_in_attribute][df[target]==value_in_target])


'outlook'

# Tree Construction

In [56]:
def buildTree(df,tree=None):
    # last column in dataframe
    target = df.keys()[-1]
    
    # Node contains the column with maxium information gain
    node = find_best_attribute_to_divide(df)
    
    # attValue contains all the unique values present in the node column of the dataset
    attValue = np.unique(df[node])
    
    # Creating an empty tree
    if tree == None:
        tree = {}
        tree[node] = {}
        
    for value in attValue:
        subtable = df[df[node] == value].reset_index(drop=True)
        clValue,counts = np.unique(subtable['play'],return_counts=True)
        
        if len(counts)==1:
            tree[node][value] = clValue[0]
        else:
            tree[node][value] = buildTree(subtable)
            
    return tree

In [67]:
buildTree(df)

C:\Users\user\AppData\Local\Temp/ipykernel_9980/246884125.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  num = len(df[df[attribute]==value_in_attribute][df[target]==value_in_target])


{'outlook': {'Overcast': 'Yes',
  'Rain': {'wind': {'Strong': 'No', 'Weak': 'Yes'}},
  'Sunny': {'humidity': {'High': 'No', 'Normal': 'Yes'}}}}